#### IMPORT MODULES

In [58]:
import cohere
import pandas as pd
import requests
import datetime
import os,sys
sys.path.append('../scr')
import config
from File_handler import FileHandler
file_data=FileHandler()
api_key = config.cohere_api["api_key"]

In [59]:
# Paste your API key here. Remember to not share publicly
#api from config

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [73]:
#data = file_data.dvc_get_data('../data/relations_dev.json', 'v3')
import json
#train data
with open(r"../data/relations_dev.json","r") as read_file:
  train_df = pd.read_json(read_file)
#test data
with open(r"../data/relations_test.json","r") as read_file:
  test_df = json.load(read_file) 

In [74]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   document   22 non-null     object
 1   tokens     22 non-null     object
 2   relations  22 non-null     object
dtypes: object(3)
memory usage: 656.0+ bytes


In [81]:
sample_train_data = train_df[:6]
sample_train_data

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP..."
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG..."
5,8+ years of development experience with Python...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."


In [82]:
from pipeline import Pipeline
pipeline =Pipeline()

In [83]:
val = pipeline.extract_values(sample_train_data)
#The first thing is to extract all the data from our json

2022-09-17 11:46:34,961:logger:string indices must be integers


In [84]:
sample_test_df = test_df[6]
sample_test_df

{'document': '\nPh.D. with 5+ years of experience, MS with 7+ years of experience, or BS with 10+ years of experience in Physics, Electrical Engineering, Computer Science, or a related technical field such us architecting, developing, and launching hardware/software projects and/or services\nDemonstrated knowledge dissemination through authored publications, international conference presentations or shipped products\nML/AI basics, and systems basics, including the requisite programming experience (python or equivalent, and at least one systems-level programming language: C, C++, Java, Go, Rust, or equivalent)\nExperience with data analytics  (data collection, storage, cleaning, processing with statistics, visualization, and other data related processes)\nExperience working on communication systems in a research and/or development capacity\nTechnical leadership in leading research efforts with a demonstrated experience handling multiple priorities\n\nPREFERRED \nDeep understanding of ho

In [85]:
test_token_dict = pipeline.get_tokens(sample_test_df)
test_token_dict

2022-09-17 11:46:38,122:logger:Tokens converted to Dict


{}

In [86]:
prompt = val
prompt = f"\nJob Description: {test_df[2]['document'].strip()}"

In [87]:
test_token_dict = pipeline.get_tokens(train_df)
test_token_dict

2022-09-17 11:46:44,038:logger:list indices must be integers or slices, not str


{}

In [88]:
response =pipeline.send_request_to_cohere(co, prompt)
response
#Send our request to cohere API

2022-09-17 11:46:58,755:logger:Rquest successful


'\nExperience with agile development methodologies\nExperience with web-based software, web-based applications, and/or APIs\nExperience with cloud computing (AWS)\n\nIf this sounds like you, apply below!\n\nhttps://recruiting.ultipro.com/INTL1090INTL/JobBoard/a0e5d4cb-e8a6-4f4b-9d6a-d2d55e7f0f3b/OpportunityDetail?opportunityId=5e7d9f2a-1f8a-4a34-a2a6-d0e79d4c7d4a\n\nThanks,\nRohit\nA:\nI have been working on a lot of these types of project management roles. I would say that this is a pretty standard set of requirements. \nI would say that the most important thing is that you can demonstrate experience in managing projects. I would focus on the most recent projects you have worked on and try to show that you were the main point of contact for the project and that you were responsible for managing the project. \nI would also focus on the skills that you have that are relevant to the job. If you have experience with agile development methodologies, mention that. If you have experience wit

In [89]:
response_dict = pipeline.process_response(response)
response_dict

{'https': '//recruiting.ultipro.com/INTL1090INTL/JobBoard/a0e5d4cb-e8a6-4f4b-9d6a-d2d55e7f0f3b/OpportunityDetail?opportunityId=5e7d9f2a-1f8a-4a34-a2a6-d0e79d4c7d4a',
 'A': ''}

In [ ]:
tokens = pipeline.get_tokens(train_df[1])
tokens

In [ ]:
with open(r"../data/Example.csv","r") as read_file:
  train_df = pd.read_json(read_file)

In [91]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
      extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor([e[1] for e in movie_examples], 
                                       [e[0] for e in movie_examples], [],
                                       "", 
                                       "extract the movie title from the post:")


NameError: name 'movie_examples' is not defined

In [ ]:
# This is what the prompt looks like:
print(cohereMovieExtractor.make_prompt('<input text here>'))

In [ ]:
num_posts = 10

movies_list = get_post_titles(size=num_posts, 
      after=str(int(datetime.datetime(2021,1,1,0,0).timestamp())), 
      before=str(int(datetime.datetime(2022,1,1,0,0).timestamp())), 
      subreddit="movies", 
      sort_type="score", 
      sort="desc")

# Show the list
movies_list

In [ ]:
results = []
for text in tqdm(movies_list):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

In [ ]:
pd.DataFrame(data={'text': movies_list, 'extracted_text': results})

In [ ]:
test_df = pd.read_csv('data/Expamle.csv',index_col=0)
test_df

In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

In [ ]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

In [ ]:
test_df[test_df['correct']==0]

In [ ]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print(classification_report(test_df['label'].str.lower(), test_df['extracted_text'].str.lower()))

In [ ]:
=============///===================